In [2]:
from neo4j import GraphDatabase

In [8]:
uri = "neo4j+ssc://ec3c7039.databases.neo4j.io"
username = "neo4j"
password = "BitmrVjrT7aAhXHF3tob9hEKgq7RfHa1LE5zKHaL554"

In [9]:
driver = GraphDatabase.driver(uri,auth=(username,password))

In [10]:
def create_node(transaction,name,age):
    transaction.run("CREATE (:Person {name:$name,age:$age})",name=name,age=age)

with driver.session() as session:
    session.execute_write(create_node,"Arturs",23)

In [13]:
def create_friendship(transaction,name1,name2):
    transaction.run("MATCH (person1:Person {name:$name1}),(person2:Person {name:$name2}) CREATE (person1)-[:FRIEND_OF]->(person2)",name1=name1,name2=name2)

In [14]:
with driver.session() as session:
    session.execute_write(create_friendship,"Arturs","Bob")

In [19]:
def get_person(transaction,name):
    result = transaction.run("MATCH (person:Person {name:$name}) RETURN person.name,person.age",name=name)
    return [{"name":record["person.name"],"age":record["person.age"]} for record in result]

In [20]:
with driver.session() as session:
    result = session.execute_read(get_person,"Bob")
    print(result)

[{'name': 'Bob', 'age': 20}]


In [21]:
def get_person_with_friends(transaction,name):
    result = transaction.run("MATCH (person:Person {name:$name})-[:FRIEND_OF]->(friend:Person) RETURN person.name,person.age,friend.name,friend.age",name=name)
    return [{"name":record["person.name"],"age":record["person.age"],"friend_name":record["friend.name"],"friend_age":record["friend.age"]} for record in result]

In [22]:
with driver.session() as session:
    result = session.execute_read(get_person_with_friends,"Alice")
    print(result)

[{'name': 'Alice', 'age': 21, 'friend_name': 'Bob', 'friend_age': 20}, {'name': 'Alice', 'age': 21, 'friend_name': 'Jessica', 'friend_age': 20}]


In [23]:
def update_age(transaction,name,age_upd):
    transaction.run("MATCH (person:Person {name:$name}) SET person.age = $age_upd",name=name,age_upd=age_upd)

In [24]:
with driver.session() as session:
    session.execute_write(update_age,"Arturs",25)

In [29]:
def get_friends_of_friends(transaction,name):
    result = transaction.run("MATCH (person:Person {name:$name})-[:FRIEND_OF]->(:Person)-[:FRIEND_OF]->(fof) WHERE fof.name <> $name RETURN fof.name,fof.age",name=name)
    return [{"name":record["fof.name"],"age":record["fof.age"]} for record in result]

In [32]:
with driver.session() as session:
    results = session.execute_read(get_friends_of_friends,"Bob")
    print(results)

[{'name': 'Jessica', 'age': 20}]


In [40]:
def get_parent_of_friend(transaction,name):
    result = transaction.run("MATCH (person:Person {name:$name})-[:FRIEND_OF]->(friend:Person)<-[:PARENT]-(pof) RETURN friend.name,friend.age,pof.name,pof.age",name=name)
    return [{"friend_name":record["friend.name"],"friend_age":record["friend.age"],"name":record["pof.name"],"age":record["pof.age"]} for record in result]

In [41]:
with driver.session() as session:
    results = session.execute_read(get_parent_of_friend,"Bob")
    print(results)

[{'friend_name': 'Alice', 'friend_age': 21, 'name': 'Jack', 'age': 50}]


In [77]:
def find_shortest_path(transaction,name1,name2):
    result = transaction.run("match (person1:Person {name:$name1}),(person2:Person {name:$name2}), path= SHORTESTPATH ((person1)-[*]->(person2)) RETURN nodes(path),relationships(path)",name1=name1,name2=name2)
    return [{"nodes":record["nodes(path)"],"relationships":record["relationships(path)"]} for record in result]

In [78]:
with driver.session() as session:
    results = session.execute_read(find_shortest_path,"Arturs","Alice")
    nodes = results[0]["nodes"]
    relationships = results[0]["relationships"]

In [79]:
for node in nodes:
    print(node.get("name"))

Arturs
Jessica
Alice


In [83]:
for relationship in relationships:
    nodes = relationship.nodes
    print(nodes[0].get("name"),"->",nodes[1].get("name"))

Arturs -> Jessica
Jessica -> Alice
